In [1]:
import BioSimSpace.Sandpit.Exscientia as BSS
import numpy as np
import pandas as pd
from pathlib import Path

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
pdb = '6rna'

# Load system

and set molecule to be decoupled

In [50]:
system = BSS.IO.readMolecules(f"{pdb}/production/final/*", make_whole=True, )

In [5]:
lig_ix = 1

In [6]:
lig = BSS.Align.decouple(system[lig_ix])
system.updateMolecule(lig_ix, lig)

In [7]:
system.nDecoupledMolecules()

1

In [8]:
decoupled_mol = system.getDecoupledMolecules()
print(decoupled_mol)

<BioSimSpace.Molecules: nMolecules=1>


## Estimate optimal restraints

In [39]:

out_dir = Path(f"{pdb}/restraint_search")
out_dir.mkdir(exist_ok=True, parents=True)

    
traj = BSS.Trajectory.Trajectory(
    trajectory=f"{pdb}/production/gromacs.xtc",
    topology=f"{pdb}/solvated/solvated.prm7",
)

# Search for the optimal Boresch restraints
restraint = BSS.FreeEnergy.RestraintSearch.analyse(
    work_dir = f"{pdb}/restraint_search",
    system = system,
    traj = traj,
    temperature = 300 * BSS.Units.Temperature.kelvin,
    method="BSS",
    restraint_type = "Boresch",
)

Indexing XTC               3.0 s 126716640.0 bytes / s          


Searching for low variance pairs. Frame no:   0%|          | 0/2501 [00:00<?, ?it/s]

Scoring candidate Boresch anchor points. Anchor set no:   0%|          | 0/50 [00:00<?, ?it/s]

In [42]:
restraint

In [43]:
restraint.getCorrection()

-11.0104 kcal/mol

In [125]:
print(restraint.toString(engine="somd"))

boresch restraints dictionary = {"anchor_points":{"r1":1448, "r2":1446, "r3":1458, "l1":4981, "l2":4980, "l3":4982}, "equilibrium_values":{"r0":5.27, "thetaA0":1.34, "thetaB0":1.78,"phiA0":-2.00, "phiB0":1.96, "phiC0":2.33}, "force_constants":{"kr":10.49, "kthetaA":51.44, "kthetaB":46.18, "kphiA":121.38, "kphiB":111.59, "kphiC":56.94}}


In [64]:
view = BSS.Notebook.View(system)
view.system()

NGLWidget(gui_style='ngl')

## Wrap coordinates for viewing restraints

The system doesn't have the ligand wrapped in the unit cell so let's wrap it and then view the restraints

In [105]:
import mdtraj as md
import nglview as nv

In [103]:
traj = md.load(f'{pdb}/production/gromacs.xtc', top=f'{pdb}/production/gromacs.gro')

In [106]:
view = nv.show_mdtraj(traj)
view

NGLWidget(max_frame=2500)

In [109]:
traj = traj.make_molecules_whole()
traj = traj.image_molecules()

In [110]:
view = nv.show_mdtraj(traj)
view

NGLWidget(max_frame=2500)

In [122]:
traj.save_xtc(f'{pdb}/production/wrapped/gromacs.xtc')

traj[0].save_pdb(f'{pdb}/production/wrapped/top.pdb')

In [123]:
system_wrapped = BSS.IO.readMolecules([f'{pdb}/production/wrapped/gromacs.xtc', 
                                       f'{pdb}/production/wrapped/top.pdb'], make_whole=True)

Indexing XTC               3.1 s 126197784.0 bytes / s          


In [126]:
view = BSS.Notebook.View(system_wrapped)
view.system()

NGLWidget(gui_style='ngl')

## Pickle restraint for use later

In [127]:
import pickle

In [128]:
pickle.dump(restraint, open(f'{pdb}/restraint_search/restraint.pkl', 'wb'))